In [1]:
# from __future__ import division
# from __future__ import print_function
import caffe
import numpy as np
import os
import sys
from datetime import datetime
from PIL import Image as pilimg
from IPython.display import Image
from IPython.display import clear_output
import time

import xml.etree.ElementTree as ET

import os.path as osp
from easydict import EasyDict as edict
import yaml

In [2]:
from matplotlib.pyplot import imshow
from matplotlib.pyplot import imread
from matplotlib.pyplot import imsave
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

In [3]:
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    if isinstance(sequence, dict):
                        val = ''
                    else:
                        val=sequence[index-1]
                            
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=val,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

In [4]:
def get_classes(classes_file_path):
    classes = []
    with open(classes_file_path, 'r') as fp:
        for line in fp:
            if line[0]=='#' or line[0]=='\n':
                pass
            else:
                classes.append(line[:-1])
    return classes

In [5]:
# seg_basic_path='/home/jogue/workspace/datasets/InOutDoorPeople/labels_mask/'
seg_basic_path='/home/jogue/workspace/datasets/InOutDoorPeople/labels_mask_person/'
depth_basic_path='/home/jogue/workspace/datasets/InOutDoorPeople/Depth/'
rgb_basic_path='/home/jogue/workspace/datasets/InOutDoorPeople/Images/'
xml_basic_path='/home/jogue/workspace/datasets/InOutDoorPeople/Annotations_pv/'
splits_basic_path = '/home/jogue/workspace/datasets/InOutDoorPeople/ImageSets/'
# classes_file_path = '/home/jogue/workspace/datasets/SUNRGBD_pv/37_classes.txt'
classes_file_path = '/home/jogue/workspace/datasets/InOutDoorPeople/classe_person.txt'

splits=['seq0.txt', 'seq1.txt',  'seq2.txt',  'seq3.txt']

classes = get_classes(classes_file_path)

#classes=['background','person']
_class_to_ind = dict(zip(classes, xrange(len(classes))))
initial_mask = len(classes)*np.ones((1080,1920),dtype=np.uint8)
mask = np.empty((1080,1920),dtype=np.uint8)

In [6]:
for split in log_progress(splits, title='Splits', end_msg='Finished !',removeAtTheEnd=False):
    list_images = open(splits_basic_path+split, 'r').read().splitlines()
    for img_name in log_progress(list_images, title='img', end_msg='Finished !',removeAtTheEnd=True):
        
        ##files
        rgb_file = rgb_basic_path+img_name+'.png'
        depth_file = depth_basic_path+img_name+'.png'
        xmlfile = xml_basic_path+img_name+'.xml'
        segfile = seg_basic_path+img_name+'.png'

        tree = ET.parse(xmlfile)
        objs = tree.findall('object')
        only_in_cls_obj = [obj for obj in objs if obj.find('name').text.lower().strip() in classes]
        objs = only_in_cls_obj
        num_objs = len(objs)
        img_rois = np.zeros((num_objs, 5), dtype=np.uint16)
        mask = len(classes)*np.ones((1080,1920),dtype=np.uint8)
        for ix, obj in enumerate(objs):
            bbox = obj.find('bndbox')
            # Make pixel indexes 0-based
            x1 = float(bbox.find('xmin').text) - 1
            y1 = float(bbox.find('ymin').text) - 1
            x2 = float(bbox.find('xmax').text) - 1
            y2 = float(bbox.find('ymax').text) - 1
            label = _class_to_ind[obj.find('name').text.lower().strip()]
            mask[int(y1):int(y2+1),int(x1):int(x2+1)]=label
        # Creates PIL image
        image = pilimg.new("L", (1080, 1920))
        image = pilimg.fromarray(mask)
        image.save(segfile)
    